# WebGPU Acceleration in Jupyter Lab
# Jupyter LabでのWebGPU加速

This notebook demonstrates WebGPU acceleration capabilities with RusTorch WASM in Jupyter Lab, specifically optimized for Chrome browsers.

このノートブックでは、Chrome ブラウザ向けに特別に最適化された、Jupyter LabでのRusTorch WASMのWebGPU加速機能を実演します。

## Prerequisites / 前提条件

**Required:** Chrome 113+ with WebGPU enabled

### Chrome Setup:
1. Open `chrome://flags/#enable-unsafe-webgpu` → Set to **Enabled**
2. Open `chrome://flags/#enable-webgpu-developer-features` → Set to **Enabled**
3. Restart Chrome
4. Verify WebGPU: Open DevTools → Console → `console.log('WebGPU:', !!navigator.gpu)`

In [ ]:
// Load and initialize RusTorch
const script = document.createElement('script');
script.src = '../rustorch_jupyter.js';
document.head.appendChild(script);
await new Promise(resolve => { script.onload = resolve; });

const rustorch = new RusTorchJupyter();
await rustorch.initialize();

console.log('🎮 WebGPU acceleration demo initialized');

## WebGPU Capability Detection / WebGPU機能検出

In [ ]:
// Check browser WebGPU support
console.log('🔍 Browser WebGPU Support Analysis:');
console.log('Navigator GPU available:', !!navigator.gpu);
console.log('User Agent:', navigator.userAgent);

if (navigator.gpu) {
    try {
        const adapter = await navigator.gpu.requestAdapter({
            powerPreference: 'high-performance'
        });
        
        if (adapter) {
            console.log('✅ WebGPU adapter found');
            console.log('Adapter info:', {
                vendor: adapter.info?.vendor || 'Unknown',
                architecture: adapter.info?.architecture || 'Unknown',
                device: adapter.info?.device || 'Unknown'
            });
            
            const device = await adapter.requestDevice();
            console.log('✅ WebGPU device created successfully');
            console.log('Device limits:', {
                maxTextureDimension2D: device.limits.maxTextureDimension2D,
                maxComputeWorkgroupsPerDimension: device.limits.maxComputeWorkgroupsPerDimension
            });
        } else {
            console.log('❌ No WebGPU adapter available');
        }
    } catch (error) {
        console.log('❌ WebGPU initialization failed:', error.message);
    }
} else {
    console.log('❌ WebGPU not supported in this browser');
}

## RusTorch WebGPU Integration / RusTorch WebGPU統合

In [ ]:
// Initialize RusTorch WebGPU
const webgpu = await rustorch.webgpu();
const utils = await rustorch.utils();

if (webgpu) {
    console.log('🚀 RusTorch WebGPU acceleration enabled!');
    
    // Test basic tensor operations
    const test_a = [1, 2, 3, 4, 5, 6, 7, 8];
    const test_b = [8, 7, 6, 5, 4, 3, 2, 1];
    
    console.log('🧮 Testing WebGPU tensor operations:');
    const add_result = webgpu.tensor_add_cpu(test_a, test_b);
    console.log('Tensor addition result:', add_result);
    
} else {
    console.log('⚠️ WebGPU not available - using CPU fallback');
    console.log('💡 To enable WebGPU: Use Chrome 113+ with WebGPU flags enabled');
}

## Performance Comparison / パフォーマンス比較

Compare WebGPU vs CPU performance for different operations.

In [ ]:
// Performance benchmarking: WebGPU vs CPU
console.log('📊 WebGPU vs CPU Performance Comparison:');

const benchmark_sizes = [1000, 5000, 10000];
const performance_results = [];

for (const size of benchmark_sizes) {
    console.log(`\n🔬 Testing size: ${size} elements`);
    
    // Generate test data
    const data_a = new Array(size).fill(0).map(() => Math.random());
    const data_b = new Array(size).fill(0).map(() => Math.random());
    
    // Create CPU tensor for comparison
    const cpu_tensor_a = await rustorch.createTensor(data_a, [Math.sqrt(size), Math.sqrt(size)]);
    const cpu_tensor_b = await rustorch.createTensor(data_b, [Math.sqrt(size), Math.sqrt(size)]);
    
    // CPU benchmark
    const cpu_time = await utils.benchmark(async () => {
        cpu_tensor_a.add(cpu_tensor_b);
    }, 50);
    
    // WebGPU benchmark (if available)
    let webgpu_time = null;
    let speedup = null;
    
    if (webgpu) {
        webgpu_time = await utils.benchmark(async () => {
            webgpu.tensor_add_cpu(data_a, data_b);
        }, 50);
        speedup = cpu_time / webgpu_time;
    }
    
    const result = {
        size,
        cpu_time: cpu_time.toFixed(2),
        webgpu_time: webgpu_time ? webgpu_time.toFixed(2) : 'N/A',
        speedup: speedup ? speedup.toFixed(2) + 'x' : 'N/A'
    };
    
    performance_results.push(result);
    
    console.log(`CPU: ${result.cpu_time}ms`);
    console.log(`WebGPU: ${result.webgpu_time}ms`);
    console.log(`Speedup: ${result.speedup}`);
}

console.log('\n📈 Performance Summary:');
console.table(performance_results);

## Matrix Multiplication Benchmark / 行列乗算ベンチマーク

In [ ]:
// Matrix multiplication performance test
console.log('🔢 Matrix Multiplication Performance Test:');

if (webgpu) {
    const matrix_sizes = [64, 128, 256];
    
    for (const size of matrix_sizes) {
        console.log(`\n📐 Testing ${size}x${size} matrix multiplication:`);
        
        // Generate random matrices
        const matrix_a = new Array(size * size).fill(0).map(() => Math.random());
        const matrix_b = new Array(size * size).fill(0).map(() => Math.random());
        
        // WebGPU matrix multiplication
        const webgpu_matmul_time = await utils.benchmark(async () => {
            webgpu.matrix_multiply_cpu(matrix_a, matrix_b, size, size, size);
        }, 10);
        
        // Calculate theoretical FLOPS
        const flops = 2 * size * size * size; // 2*N^3 for matrix multiplication
        const gflops = (flops / (webgpu_matmul_time / 1000)) / 1e9;
        
        console.log(`WebGPU time: ${webgpu_matmul_time.toFixed(2)}ms`);
        console.log(`Performance: ${gflops.toFixed(2)} GFLOPS`);
    }
} else {
    console.log('⚠️ WebGPU not available - matrix multiplication benchmarks skipped');
    console.log('💡 Enable WebGPU in Chrome for GPU-accelerated matrix operations');
}

## Jupyter Lab Integration Summary / Jupyter Lab統合まとめ

### ✅ Completed Features:
- **WASM Module Loading**: Automatic initialization with fallback handling
- **WebGPU Acceleration**: Chrome-optimized GPU computing with CPU fallback
- **Tensor Operations**: Full tensor API with Jupyter-friendly display
- **Performance Benchmarking**: Built-in benchmarking utilities
- **Error Handling**: Jupyter-specific error management

### 🎯 Key Benefits:
- **Interactive Development**: Real-time ML experimentation in browser
- **No Server Required**: Client-side ML with WASM performance
- **GPU Acceleration**: Hardware acceleration where supported
- **Educational**: Perfect for learning and teaching ML concepts

### 🚀 Next Steps:
- Experiment with larger datasets and models
- Try different neural network architectures
- Explore statistical analysis workflows
- Test performance on your specific hardware